### IllumiDesk Tips



* Within code cells, click the blue play button on the left to run the code.
* In the upper right, you can see if your kernel is connected. That is a dropdown menu that has additional functionality to “disconnect”, “restart”, and “clear all outputs” of the Kernel. None of these buttons will delete your work.
* If you would like to test code, then you can access the console at the bottom of the page and play with code there.
* Do not forget that code runs sequentially!
* Selecting the clipboard icon on the right will allow you to skip to different parts of this document without scrolling.




---



### Introduction to While Loops



Julia Documentation: [Control Flow · The Julia Language](https://docs.julialang.org/en/v1/manual/control-flow/)



The first thing that happens when a `while` loop is executed is that the controlling Boolean expression is evaluated. If the Boolean expression evaluates to `false`, then the body of the loop is never executed. It might seem pointless to execute the body of a loop zero times, but that is sometimes the desired action.



For example, a `while` loop is often used to sum a list of numbers, but the list could be empty. To be more specific, a checkbook balancing program might use a `while` loop to sum the values of all checks you have written in a month, but you might take a month’s vacation and write no checks at all. In that case, there are zero numbers to sum, so the loop is iterated zero times.



The body of a `while` loop should change the value of one or more global variables so that the condition becomes `false` eventually and the loop terminates. Otherwise, the loop will repeat forever, which is called an **infinite loop**.



In [1]:
#General Format of a While Loop

while Boolean\_Expression\_Here
 #body of the loop
end

UndefVarError: UndefVarError: `Boolean` not defined

In [2]:
using LinearAlgebra

# Example: Sum of the entries in v

v = [1;2;3]

#=
Contrary to a for loop, the while loop conditions can not be initalized 
inside of the loop header. The variables must be declared outside.
=#

# Notice i is declared in the header
sumFor = 0
  for i in 1:length(v)
  sumFor += v[i]
end
@show sumFor


#=
Notice j is declared outside of the header

Notice that inside of the loop, we have to increase j.

- j will not increase itself in the same way that i does in the
for loop above.

- if I do not increase j, then the while loop will run forever
=#
sumWhile = 0
j = 1
while j <= length(v)
  sumWhile += v[j]
  j += 1 
end

#=
If this @show statement is not here
then we can not expect to see any output
of any variables updated within the loop
=#
@show sumWhile 

sumFor = 6
sumWhile = 6


6

A `while` loop is used as opposed to a `for` loop when we are unsure of the number of iterations required to complete the task.



The Bisection Algorithm is a great example to show when a `for` loop could NOT be used, and a `while` loop is necessary. Although we may know the starting condition for the bisection algorithm, we do not know the ending condition which is required in a `for` loop. We don't know whether we need to loop 5 times or 100 times. However, we do know that we need to stop when we've found a root or when too many iterations have taken place. The latter prevents us from writing an infinite loop.



### Break



Sometimes, you don’t know it’s time to end a loop until you get halfway through the body. In that case, you can use the `break` statement to jump out of the loop.



In [ ]:
#Break Example
i = 0
while true 
    i = i + 1
    if i == 6
        break
    else
        println(i) 
    end 
end 



---



**Symbolic, Numerical, and Automatic Differentiation**
======================================================



**& the Glories of the Jacobian**
=================================



**In this lab, we take a look at various ways of computing derivatives of mathematical functions**



**and make a robot dance using a Jacobian.**



In [2]:
using Symbolics, LinearAlgebra, FiniteDiff, ForwardDiff, Plots
gr()

Plots.GRBackend()



---



Symbolic Differentiation
------------------------



This is the kind of differentiation you learn (or will learn) in your intro Calculus classes. Using this approach, you can take derivatives of expressions using the various rules of differential calculus, such as:



\* \frac{d}{d x} c = 0 *where* c *is a numeric constant*



### \*\frac{d}{d x} x = 1



\* \frac{d}{d x} y = 0 *where* y *is a variable other* *than* x



### \*\frac{d}{dx} (u + v) = \frac{d}{d x} u + \frac{d}{d x} v = 0



### *\**\frac{d}{dx} (u \cdot v) = u \cdot \frac{d}{d x} v + v \cdot \frac{d}{dx} u



### *\**\frac{d}{d x} \frac{u}{v} = \frac{(v \* \frac{d}{d x}u - u \* \frac{d}{dlx} v)}{v^2}



\*…and so on



This form of differentiation is called symbolic, because it allows you to perform differentiation by manipulating symbols, or more precisely, variables, without actually evaluating them real numbers. This is a big part of Calc I.



As you may have guessed, Julia has a sweet package called `Symbolics.jl` which allows you to take symbolic derivatives of input expressions. The next few cells will teach you how to use this package.



### Example 1



Find the derivative of \frac{df}{dx} of the function f(x) = x³ cos(X) + x² sin(x)² + log(3x) using symbolic differentiation.



In [ ]:
#=
@variables is a macro
that declares the variables
in the function
=#
@variables x

#differential operator 
d╱dx = Differential(x)

#define f(x) 
f = x^3*cos(x) + x^2*sin(x)^2 + log(3x)

#apply the differential operator
@show d╱dx(f)

#= 
compute the partial derivative
using the expand_derivatives function
=#
@show dfdx = expand_derivatives(d╱dx(f))

#=
evaluate the derivative expression at some value
using the substitute function

x = 0.5
=#
@show val = substitute(dfdx, (Dict(x=>0.5)))



---



Numerical Differentiation
-------------------------



Numerical differentiation is another approach for taking derivatives of functions. Unlike Symbolic differentiation where we derive exact, analytical derivatives of functions, Numerical differentiation estimates the derivatives of functions using values of the function at different points.



The simplest and most common method for Numerical Differentiation is **Finite Difference**.



Finite Difference approximates the derivative \frac{\partial f}{\partial x} of a function f at some numerical point x\_0 as \frac{\partial f(x\_0)}{\partial x} = \frac{f(x\_0+h) - f(x\_0)}{h}



This finite difference variety is specifically known as the Forward Difference approximation. There also exists a Backward Difference approximation which is expressed as \frac{\partial f(x\_0)}{\partial x} = \frac{f(x\_0) - f(x\_0-h)}{h} and Symmetric Difference approximation expressed as \frac{\partial f(x\_0)}{\partial x} = \frac{f(x\_0+h) - f(x\_0-h)}{2h}.



If the derivative off(x) exists at point x\_0, for some small h, the forward, backward and symmetric difference approximations are approximately equal. If they end up being significantly different, then the function f(x) is not differentiable.



As you would have guessed, there exists a really sweet Julia package with implementations of finite differences called `FiniteDiff.jl`. `FiniteDiff.jl` has empirically the fastest implementation of finite differences in any programming language!



In the following cells, we are going to learn how to use it to take derivatives, gradients, Jacobians and Hessians of functions.



The main difference between derivatives and gradients in relation to the `FiniteDiff.jl` package is that, derivatives are used for functions with scalar inputs and gradients are used for functions with vector inputs.



### Example 2



Find the derivative of \frac{df}{dx} of the function f(x) = x³ cos(X) + x² sin(x)² + log(3x) using finite differences.



In [ ]:
# define function
g(x) = x^3*cos(x) + x^2*sin(x)^2 + log(3x)

#=
derivative of g at x = 0.5
using the function 
FiniteDiff.finite_difference_derivative

First argument is the function
Second argument is the point of interest
=#
val = FiniteDiff.finite_difference_derivative(g, 0.5)

# Solution almost equal to Symbolic from Ex. 1!

### Example 3



Compute the Jacobian of the function below using finite differences where x \in \mathbb{R}³ and x\_i is the i-th element in x



h(x) = \begin{bmatrix} x\_1^3cos(x\_2) + x\_2^2sin(x\_1)^2 \\ log(3x\_3) \end{bmatrix}



In [ ]:
#define the function
h(x) = [x[1]^3*cos(x[2]) + x[2]^2*sin(x[1])^2;
                        log(3x[3])]

# the point of interest
x₀ = [0.1, 0.5, 0.7]

#Jacobian
J = FiniteDiff.finite_difference_jacobian(h, x₀)

#Gradients & Hessians can be computed similarly



---



Automatic Differentiation
-------------------------



Similar to Symbolic Differentiation, Automatic Differentiation uses differential calculus rules to compute derivatives. However, its goal is to return a numerical solution instead of a single closed-form analytical expression. As such, Automatic Differentiation keeps track of intermediate variables and their derivatives to speed up computation. It decomposes the function into primitive operations using the chain rule, evaluates the operations and their derivatives and stores them in intermediate variables.



To appreciate how automatic differentiation works, let's take a look at an example.



We would like to compute the Jacobian of the multivariate function:



f(x,y) = y\cdot sin(x) + y^2



at a point x=0.1, y=0.3 using automatic differentiation.



First, let us define the (^\bullet)\_x operator as the derivative with respect to x, \frac{\partial}{\partial x}. The notation \dot{a} for the derivative is due to Isaac Newton <https://en.wikipedia.org/wiki/Notation_for_differentiation>



Next, we will decompose f(x,y) into primitive operations and store their values and derivatives with respect to x in intermediate variables.



(View **dx-5.png** in the attachments) This routine is called a **forward pass** with respect to x. It is worth noting that, unlike symbolic differentiation, the numerical values of each intermediate variable is computed in each row of the table.



Similarly, we define the (^\bullet)\_y operator as the derivative with respect to y, \frac{\partial}{\partial y} and define the intermediate variables and derivatives with respect to y



(View **dy.png** in the attachments)



The Jacobian J of f(x,y) can now be defined as J = [\dot{d}\_x~~~\dot{d}\_y] = [0.2985 ~~~ 0.6998]



The computational routine we have just demonstrated above is a special kind of Automatic Differentiation called Forward-Mode Automatic Differentiation.



In general, you would perform a forward pass for each argument to your function f. If the function argument is a vector, you would perform a forward pass for each element in the vector. The computational complexity is of the order O(n). You can imagine that, this could end up being computationally inefficient for functions f : \mathbb{R}^n \rightarrow \mathbb{R}^m with n >>> m. In such cases, another special kind of Automatic Differentiation called Reverse-mode Automatic Differentiation will be much more appropriate.



Reverse-mode Automatic Differentiation first performs a forward pass to compute **only** the intermediate variable values without taking the derivatives. It then performs a reverse (backwards) pass to compute the derivatives of the intermediate variables. A function f : \mathbb{R}^n \rightarrow \mathbb{R}^m, will require m reverse passes to compute the Jacobian of the function. Because of this, Reverse-Mode automatic differentiation is suitable for functions where n >>> m and inefficient for functions were m >>> n.



As you may rightly guess, Julia has packages that have super-fast implementations of forward-mode and reverse-mode automatic differentiation named `ForwardDiff.jl` and `ReverseDiff.jl`.



In the following cells, we will demonstrate the usage of `ForwardDiff.jl` for computing derivatives and Jacobians.



### Example 4



Find the derivative \frac{d f\_2}{d x} of the function f\_2(x) = x^3cos(x) + x^2sin(x)^2 + log(3x) using Forward-mode Automatic Differentiation.



In [ ]:
#define function
f₂(x) = x^3*cos(x) + x^2*sin(x)^2 + log(3x)

#=
Evaluate derivative at x =0.5
using the ForwardDiff.derivative function

First argument is function
Second argument is the pointof interest
=#
x = 0.5
df₂╱dx = ForwardDiff.derivative(f₂, x)

#Same result as Symbolic! 

### Example 5



Compute the Jacobian of the function below using Forward-mode Automatic Differentiation where x \in \mathbb{R}^3 and x\_i is the i-th element in x



h\_2(x) = \begin{bmatrix} x\_1^3cos(x\_2) + x\_2^2sin(x\_1)^2 \\ log(3x\_3) \end{bmatrix}



In [ ]:
#define the function
h₂(x) = [x[1]^3*cos(x[2]) + x[2]^2*sin(x[1])^2;
                        log(3x[3])]

#point of interest
x₀ = [0.1, 0.5, 0.7]

#Jacobian
J = ForwardDiff.jacobian(h₂, x₀)



---



### **Advantages & Disadvantages of Each Differentiation**





| Differentiation | Advantages |
| --- | --- |
| Symbolic | * Symbolic Differentiation allows you to compute exact derivatives up to machine precision.
* Symbolic Differentiation is also a great tool for analysis. Being able to generate closed-form derivatives of a model facilitates the understanding of the dynamics of the model.
* Symbolic Differentiation is also significantly useful for generating closed-form derivatives of functions as code. Closed-loop controllers running on real-time systems often have to run at frequencies in the 1kHz range. This constrains the control loop to a time budget of 1 millisecond. Given this time budget, operations in the controller (such as computing derivatives and Jacobians) have to be as fast as possible. Being able to generate closed-form parameterized functions for derivatives and Jacobians offline using Symbolic differentiation that can then be evaluated online in microseconds can be very useful in speeding up your controller and meeting the controller time budget.
 |
| Numerical | * Numerical differentiation is the simplest differentiation method to implement. It only depends on the function values and not on any set of differential calculus rules.
* In view of this, numerical differentiation can be applied to any differentiable function to get reasonably accurate results.
* 
 |
| Automatic | * Automatic Differentiation is very fast at computing derivatives. Because of this, automatic differentiation is the primary differentiation tool for computing derivatives when training deep neural networks.
* Unlike with Finite Differences, Automatic Differentiation computes exact derivatives to machine precision.
* Implementations of Automatic Differentiation in popular Deep Learning frameworks like Pytorch and Tensorflow build *Computational Graphs* when performing the forward pass. These computational graphs allow for easy debugging and analysis.
 |





| Differentiation | Disadvantages |
| --- | --- |
| Symbolic | The main disadvantage of symbolic derivatives is that, they tend to get gnarly and exponentially long very quickly. This phenomenon is called *Expression swell*. Imagine taking the derivative of a function h(x) = f(x)g(x) where f(x) is itself the product f(x) = u(x)v(x), the derivative of h(x) ends up as this gnarly function \frac{d}{dx}h(x) = \Bigg(\frac{d}{dx}u(x)v(x) + u(x)\frac{d}{dx}v(x)\Bigg)g(x) + u(x)v(x)\frac{d}{dx}g(x) leading to inefficient code. |
| Numerical | * Finite Difference suffers from *Truncation error* resulting in inaccurate derivates when the step size is large.
* Finite Difference can also be expensive to compute for functions with large input dimensions.
 |
| Automatic | Automatic Differentiation is not as easy to implement as Finite Differences. Care usually has to be taken to implement it in a manner that is memory-efficient, particularly for reverse-mode automatic differentiation. Regardless, there are super-fast and clean implementations of automatic differentiation like [ForwardDiff.jl](https://juliadiff.org/ForwardDiff.jl/stable/) and [Zygote.jl](https://fluxml.ai/Zygote.jl/latest/) that exist off-the-shelf and can be applied to any use. |





---



**A bunch of helper code for the next section!**



In [ ]:
mutable struct Problem 
    tasks::Vector{Symbol} 
    sys
end

mutable struct Keypoint
    head_pose::Union{Nothing, Vector}
    lefthand_pose::Union{Nothing, Vector} 
    righthand_pose::Union{Nothing, Vector} 
    waist_pose::Union{Nothing, Vector}
    frequency::Int
end 


mutable struct PickleRick 
    θ::Vector{Float64}
    θ̇ ::Vector{Float64} 
    body_observables
    robot_keypoint_observables 
    obstacle_observables
    tail_observable
    dynamic::Bool 
    trail::Bool
    show_contacts::Bool 
    show_com::Bool 
    show_support_polygon::Bool
    show_goal::Bool
    show_obstacle::Bool
    l0::Float64
    l1::Float64 
    l2::Float64 
    l3::Float64 
    l4::Float64
    l5::Float64 
    l6::Float64
    l7::Float64 
    l8::Float64
    l9::Float64 
    l10::Float64
    w::Float64
    g::Union{Nothing, Vector{Vector{Float64}}}
    task_maps::Vector{Symbol}
    θᵣ::Vector{Float64}
    Δt::Float64
    m_links::Float64 
    m_head::Float64
    com_observable
    priorities::Dict
    max_range::Float64
    keypoints::Vector{Keypoint}
    kpindex::Int

    function PickleRick(joint_positions, 
                        joint_velocities, goal_position, keypoints)
        time_step = 10E-4
        task_maps = [:posture, :repeller, :lbalance, :rbalance, :lefthand_attractor]
        θᵣ = [π/2, 2π/3, π/6, 2π/3, π/6, π/2, π/3, 7π/12, 2π/3, 5π/12]
        m_links = 0.05 #kg
        m_head = 0.1 #kg
        max_range = 10.0
        priorities= Dict(:lefthand_attractor=>1,
                         :posture=>2,
                         :lbalance=>1,
                         :rbalance=>1,
                         :repeller=>2
        )
    

        new(joint_positions, 
            joint_velocities, nothing, nothing, nothing, nothing, 
            false, false, false, false, false, false, true, 0.75, 1.0, 
            1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, goal_position, 
            task_maps, θᵣ, time_step, m_links, m_head, nothing, priorities,
            max_range, keypoints, 1)
    end
end

## Kinematics
# foot as reference
function link_poses(θ, sys::PickleRick)
    l1 = sys.l1
    l2 = sys.l2
    l3 = sys.l3
    l4 = sys.l4
    l5 = sys.l5
    l6 = sys.l6
    l7 = sys.l7
    l8 = sys.l8
    l9 = sys.l9
    l10 = sys.l10
    l0 = sys.l0
    w = sys.w 
    x8 = -w
    y8 = 0.0
    x10 = w 
    y10 = 0.0
    x7 = x8 + l8 * cos(θ[8])
    y7 = y8 + l8 * sin(θ[8])
    x9 = x10 + l10 * cos(θ[10])
    y9 = y10 + l10 * sin(θ[10]) 
    x6 = 0.5*(x7 + l7 * cos(θ[7]) + x9 + l9 * cos(θ[9]))
    y6 = 0.5*(y7 + l7 * sin(θ[7]) + y9 + l9 * sin(θ[9]))
    x1 = x6 + l1 * cos(θ[6])
    y1 = y6 + l1 * sin(θ[6])
    x2 = x1 - l2 * sin(θ[2])
    y2 = y1 + l2 * cos(θ[2])
    x3 = x2 - l3 * sin(θ[3])
    y3 = y2 + l3 * cos(θ[3])
    x4 = x1 + l4 * sin(θ[4])
    y4 = y1 + l4 * cos(θ[4])
    x5 = x4 + l5 * sin(θ[5])
    y5 = y4 + l5 * cos(θ[5])
    x0 = x1 + l0 * cos(θ[1])
    y0 = y1 + l0 * sin(θ[1])
    xn = x1 + 0.5*l0 * cos(θ[1])
    yn = y1 + 0.5*l0 * sin(θ[1]) 

    return [[[x6, y6], [x7, y7], [x8, y8]], [[x6, y6], [x9, y9], [x10, y10]], 
    [[x6, y6], [x1, y1], [x0, y0]], [[x1, y1], [x4, y4], [x5, y5]], [[x1, y1], [x2, y2], [x3, y3]], [[xn, yn]]]
end

function left_hand_pose(θ, sys::PickleRick)
    poses = link_poses(θ, sys::PickleRick)
    return poses[5][end]
end

function right_hand_pose(θ, sys::PickleRick)
    poses = link_poses(θ, sys::PickleRick)
    return poses[4][end]
end

function left_foot_pose(θ, sys::PickleRick)
    poses = link_poses(θ, sys::PickleRick)
    return poses[1][end]
end

function right_foot_pose(θ, sys::PickleRick)
    poses = link_poses(θ, sys::PickleRick)
    return poses[2][end]
end

function head_pose(θ, sys::PickleRick)
    poses = link_poses(θ, sys::PickleRick)
    return poses[3][end]
end

function neck_pose(θ, sys::PickleRick)
    poses = link_poses(θ, sys::PickleRick)
    return poses[6][end]
end

function waist_pose(θ, sys::PickleRick)
    poses = link_poses(θ, sys)
    return poses[5][1]
end

function compute_COM(θ, env::PickleRick)
    l1 = env.l1
    l2 = env.l2
    l3 = env.l3
    l4 = env.l4
    l5 = env.l5
    l6 = env.l6
    l7 = env.l7
    l8 = env.l8
    l9 = env.l9
    l10 = env.l10
    l0 = env.l0
    w = env.w 
    x8 = -w
    y8 = 0.0
    x10 = w 
    y10 = 0.0
    x7 = x8 + l8 * cos(θ[8])
    y7 = y8 + l8 * sin(θ[8])
    x9 = x10 + l10 * cos(θ[10])
    y9 = y10 + l10 * sin(θ[10]) 
    x6 = 0.5*(x7 + l7 * cos(θ[7]) + x9 + l9 * cos(θ[9]))
    y6 = 0.5*(y7 + l7 * sin(θ[7]) + y9 + l9 * sin(θ[9]))
    x1 = x6 + l1 * cos(θ[6])
    y1 = y6 + l1 * sin(θ[6])
    x2 = x1 - l2 * sin(θ[2])
    y2 = y1 + l2 * cos(θ[2])
    x3 = x2 - l3 * sin(θ[3])
    y3 = y2 + l3 * cos(θ[3])
    x4 = x1 + l4 * sin(θ[4])
    y4 = y1 + l4 * cos(θ[4])
    x5 = x4 + l5 * sin(θ[5])
    y5 = y4 + l5 * cos(θ[5])
    x0 = x1 + l0 * cos(θ[1])
    y0 = y1 + l0 * sin(θ[1])

    xcm = (sum([env.m_links*x for x in [x1, x2, x3, x4, x5, x6, x7, x8, x9, x10]]) + env.m_head*x0) / (10*env.m_links + env.m_head)
    ycm = (sum([env.m_links*y for y in [y1, y2, y3, y4, y5, y6, y7, y8, y9, y10]]) + env.m_head*y0) / (10*env.m_links + env.m_head)
    
    return (xcm, ycm)
end


## Simulation
function step_plots!(θ̈ ::Vector{Float64}, prob::Problem, sys::PickleRick)
    θ = sys.θ + θ̈ *sys.Δt
    θ̇ = θ̈   
    vellims = 10.0*ones(length(θ))
    θ̇  = clamp.(θ̇ , -vellims, vellims)

    sys.θ = θ
    sys.θ̇ = θ̇ 
end


## Visualization
function visualize_plots!(sys::PickleRick)

    θ = sys.θ 
    chains = link_poses(θ, sys)
    chain1 = [[a[1], a[2]] for a in chains[1]]
    chain2 = [[a[1], a[2]] for a in chains[2]]
    chain3 = [[a[1], a[2]] for a in chains[3]]
    chain4 = [[a[1], a[2]] for a in chains[4]]
    chain5 = [[a[1], a[2]] for a in chains[5]]
    head =   [[chains[3][3][1], chains[3][3][2]]]
    Plots.plot() 
    Plots.plot!([c[1] for c in chain1], [c[2] for c in chain1] ; 
        linewidth=5, color=:purple)
    Plots.plot!([c[1] for c in chain2], [c[2] for c in chain2]; 
        linewidth=5, color=:purple)
    Plots.plot!([c[1] for c in chain3], [c[2] for c in chain3]; 
        linewidth=5, color=:purple)
    Plots.plot!([c[1] for c in chain4], [c[2] for c in chain4]; 
        linewidth=5, color=:purple)
    Plots.plot!([c[1] for c in chain5], [c[2] for c in chain5]; 
        linewidth=5, color=:purple)

    Plots.scatter!([c[1] for c in chain1], [c[2] for c in chain1], 
        color=:black, markersize=8)
    Plots.scatter!([c[1] for c in chain2], [c[2] for c in chain2], 
        color=:black, markersize=8)
    Plots.scatter!([c[1] for c in chain3], [c[2] for c in chain3], 
        color=:black, markersize=8)
    Plots.scatter!([c[1] for c in chain4], [c[2] for c in chain4], 
        color=:black, markersize=8)
    Plots.scatter!([c[1] for c in chain5], [c[2] for c in chain5], 
        color=:black, markersize=8)
    fig = Plots.scatter!([c[1] for c in head], [c[2] for c in head], color=:black, 
        markersize=20, legend=false, grid=false, aspect_ratio=:equal, 
        ylims=(-Inf, 4),
        border=:none) 
    return fig
end

## Control 
function posture_task_map(θ, prob::Problem)
    sys = prob.sys
    g = sys.θᵣ
    return θ - g
end

function lefthand_attractor_task_map(θ, prob::Problem)
    sys = prob.sys
    x₉ = sys.keypoints[sys.kpindex].lefthand_pose
    xh = left_hand_pose(θ, sys)
    return xh - x₉
end

function righthand_attractor_task_map(θ, prob::Problem)
    sys = prob.sys
    x₉ = sys.keypoints[sys.kpindex].righthand_pose
    xh = right_hand_pose(θ, sys)
    return xh - x₉
end

function head_attractor_task_map(θ, prob::Problem)
    sys = prob.sys
    x₉ = sys.keypoints[sys.kpindex].head_pose
    xh = head_pose(θ, sys)
    return xh - x₉
end

function waist_attractor_task_map(θ, prob::Problem)
    sys = prob.sys
    x₉ = sys.keypoints[sys.kpindex].waist_pose 
    xh = head_pose(θ, sys)
    return xh - x₉
end

function dodge_task_map(θ, prob::Problem)
    sys = prob.sys
    os = sys.obstacle_observables
    rs = sys.r
    chains = link_poses(θ, sys)
    xs = Float64[]; o = os[1]; r = 2*rs[1]
    lh = chains[5][end]; rh = chains[4][end]
    lẍ= chains[1][end]; rẍ= chains[2][end]
    hd = chains[3][end]; nk = chains[6][1]
    kp = [lh, rh, hd, nk]#[hd, lh, rh, lf, rf]
    for k in kp 
        Δ = (norm(k - o.val)/r)[1] - 1.0
        push!(xs, Δ)
    end 
    # @show xs
    return xs
end

### Potentials 
function posture_potential(x, ẋ, prob::Problem)
    sys = prob.sys
    M = 1.6
    K = M/3
    ϕ(x) = 0.5*x'*K*x
    δₓ = FiniteDiff.finite_difference_gradient(ϕ, x)
    f = -K*δₓ
    return f
end

function lefthand_attractor_potential(x, ẋ, prob::Problem)
    sys = prob.sys
    M = 1.6
    K = 2*M
    ϕ(x) = 0.5*x'*K*x
    δₓ = FiniteDiff.finite_difference_gradient(ϕ, x)
    f = -K*δₓ
    return f
end

function righthand_attractor_potential(x, ẋ, prob::Problem)
    sys = prob.sys
    M = 1.6
    K = 2*M
    ϕ(x) = 0.5*x'*K*x
    δₓ = FiniteDiff.finite_difference_gradient(ϕ, x)
    f = -K*δₓ
    return f
end

function head_attractor_potential(x, ẋ, prob::Problem)
    sys = prob.sys
    M = 1.6
    K = 3*M
    ϕ(x) = 0.5*x'*K*x
    δₓ = FiniteDiff.finite_difference_gradient(ϕ, x)
    f = -K*δₓ
    return f
end

function waist_attractor_potential(x, ẋ, prob::Problem)
    sys = prob.sys
    M = 1.6
    K = 3*M
    ϕ(x) = 0.5*x'*K*x
    δₓ = FiniteDiff.finite_difference_gradient(ϕ, x)
    f = -K*δₓ
    return f
end

function dodge_potential(x, ẋ, prob::Problem)
    sys = prob.sys 
    M = 1.6
    K = M 
    s = [v > sys.max_range ? 0.0 : 1.0 for v in x]  
    ϕ(σ) = (K/2) .* s .* (sys.max_range .- σ)./(sys.max_range .* σ).^2
    δₓ = FiniteDiff.finite_difference_jacobian(ϕ, x) 
    f =-K*diag(δₓ)  
    return f
end


### Solve 
function potential_eval(x, ẋ, name::Symbol, prob::Problem)
    ϕ = eval(Symbol(name, :_potential))
    f = ϕ(x, ẋ, prob)
    return f
end
 

function dance(solve_potential, keypoints; horizon=3000)
    init_joint_positions = [π/2, 2π/3, π/6, 2π/3, π/6, π/2, 
                            π/3, 7π/12, 2π/3, 5π/12]
    sys = PickleRick(init_joint_positions,
                    zero(init_joint_positions),
                    nothing, keypoints)
    sys.Δt = 5e-3
    @show sys.Δt
    tasks = [:waist_attractor, :lefthand_attractor, :righthand_attractor]
    prob = Problem(tasks, sys)
    θ = init_joint_positions
    θ̇ = zero(init_joint_positions)
    id=0
    anim = @animate for i=1:horizon  
        for (j,kp) in enumerate(sys.keypoints)
            if i%kp.frequency == 0 
                ind = (id%length(sys.keypoints))+1
                sys.kpindex = ind 
                id +=1
            end
        end
        τ = solve_potential(θ, θ̇ , prob, sys)
        step_plots!(τ, prob, sys)
        θ = prob.sys.θ; θ̇ = prob.sys.θ̇
        visualize_plots!(sys) 
    end
    return anim
end



---



The Glories of the Jacobian
---------------------------



What exactly do we use Jacobians for? We can use Jacobians to make robots dance!



![](https://i.gifer.com/origin/7e/7ebb93d3dbda6e135782626385b2e10c_w200.gif)

Math Dances Moves Video: <https://youtube.com/shorts/AHSlvTfsmCk?feature=shared>



Before we start talking about dancing robots, let's first understand a few things about Jacobians.



From our discussions of differentiation approaches so far, we know that Jacobians are essentially matrices of partial derivatives of a function. For a function:



f(x\_1, x\_2, \dots, x\_n) = [y\_1, y\_2, \dots, y\_m]



the Jacobian J can be expressed as:



J = \begin{bmatrix} \frac{\partial y\_1}{\partial x\_1} & \frac{\partial y\_1}{\partial x\_2} & \dots & \frac{\partial y\_1}{\partial x\_n} \\ \frac{\partial y\_2}{\partial x\_1} & \frac{\partial y\_2}{\partial x\_2} & \dots & \frac{\partial y\_2}{\partial x\_n} \\ \dots & \dots & \dots & \dots \\ \frac{\partial y\_m}{\partial x\_1} & \frac{\partial y\_m}{\partial x\_2} & \dots & \frac{\partial y\_m}{\partial x\_n} \\ \end{bmatrix}



Staring at the Jacobian expression above, you'd notice that number of columns of the Jacobian matrix is equal to the dimension of the input vector of function f and the number of rows of the Jacobian matrix is equal to the dimension of the output vector. The Jacobian, in essence, tells us how a small change in the inputs of the function, affects the outputs of the function. The Jacobian is a linear map from the change in inputs of the function to the change in outputs of the function. And even cooler is the fact that, the pseudo-inverse of the Jacobian maps a small change in the outputs of a function to a small change in the inputs of the function. Let's look at some math that succinctly illustrates these ideas.



\Delta y = J \cdot \Delta x



\Delta x = J^\dagger \Delta y



J^\dagger = J^\top \cdot (JJ^\top)^{-1}



where y is the vector of outputs of function f, x is the vector of inputs of function f, J is the Jacobian of function f and J^\dagger is the right Moore-Penrose pseudo-inverse of J. [For future reference, J^\dagger = J^\top \cdot (JJ^\top)^{-1} is also simply called a pseudo right inverse. We used it for solving underdetermined equations earlier in the term.]



Having inherited the power of Jacobians, let us now use it for motion control.



An important function in robot motion control is the task map. A task map is a smooth, differentiable function that maps from the robot's configuration space to a pre-defined task space for a particular task. The robot's configuration space is the space of joint positions of the robot whilst the task space could be the space of the robot's end effector cartesian coordinates if the task is a hand or foot placement task, the space of center of mass positions if the task is a center of mass placement task, or even the space of joint configurations again if the task is a robot posture task. Tasks, task spaces and task maps are up to the robot operator to design.



Once these task maps are designed, the Jacobian of the task map will now allow us to map desired motions in the task space (which are intuitive to specify) to the required motions in the robot's configuration space (which are usually unintuitive to specify). This is the power of Jacobians!



To see this in action, let's write some code!



In [ ]:
kp1 = Keypoint(nothing, [-1.36, 3.2], [1.36, 3.2], [0.0,3.5], 40)
kp2 = Keypoint(nothing, [1.36, 3.2], [-1.36, 3.2], [0.0, 2.5], 20) 

keypoints = [kp1, kp2]

We describe our dance motions using keypoints. Keypoints are the desired 2D cartesian positions. These positions are target positions to which we would like to move the robot's end effectors. The motions generated from moving the robot's end effectors to these keypoints are what will describe our dance motions.



The cell below shows the keypoints we describe for a particular dance routine.



The signature for describing a Keypoint is



`Keypoint = (head_pose, lefthand_pose, righthand_pose, waist_pose, period)`



The `period` is an integer that describes how often the keypoint is repeated in the dance routine.



#### **Here's where the magic happens.**



In order to drive the robot's end effectors to the keypoints, we will need to apply a force to the robot's end effectors. How do we compute this force? We will use one of the coolest ideas in modern robotics called Artificial Potential Fields.



Artificial potential fields are inspired by Potential Energy from Physics literature. From AP Physics, we know that Potential Energy is the kind of energy possessed by a body by virtue of its position relative to the other objects. For instance, a ball held at some height above the ground has a gravity potential energy proportional to its vertical position relative to the ground (i.e. its height). The force of gravity that pulls the object towards the ground can be computed by taking the negative derivative of the ball's gravitational potential energy with respect to its height.



Artificial potential fields behave in a similar manner. If we could build an artificial potential energy between a point of interest on a robot and a target position in space, we could compute the force needed to reach the target position by taking the negative derivative of this potential field. This idea provides as with an elegant and convenient approach for generating driving forces. To see how this is implemented in Julia, check out the accompanying `helper.jl` file.



Once we have generated the end-effector force, how do we figure out the right torques to apply to the joints of the robot to realize this force? Note that, the only way we are able to control our robot is through applying torques to its joints.



This is where the glory of the Jacobian shines!



We know that Jacobians allow us to map small changes from one space to small changes to another space. Since our force is in the end-effector space, or more formally, the task space, we can use our Jacobian to map this force to the configuration space, which is where the joints of the robot reside. All we have to do is to describe a task map that relates the robot's end effector to its target keypoint, compute the Jacobian of this task map, compute the force required to drive the end effector to the target keypoint and map that force to the robot's joint torques using the computed Jacobian. And voila, our robot will begin to dance!



For brevity sake, we describe all the task maps and potential fields in the accompanying `helper.jl` file. Check it out for more details.



The function, `potential_solve`, below describes our routine for computing joint torque \tau .



In a for loop,



\* We loop through all the task maps for all the end-effectors of the robot (lefthand, righthand, waist).



\* We compute the task space coordinate x which in this case is the difference between the current end effector position and its corresponding target keypoint.



\* Next, we compute the Jacobian of the task map using Forward-mode Automatic Differentiation (we know all about this now)



\* We then compute the end-effector force needed to reach the target waypoint using artificial potential fields.



\* Having computed the Jacobians and forces for all the task maps, we compute the final joint torques by summing up the contributions of joint torques from the individual task maps computed using \tau = J^\top\cdot f where J^\top is the Jacobian transpose which approximates the Jacobian pseudo-inverse J^\dagger and is faster to compute.



For future reference, J^\dagger = J^\top \cdot (JJ^\top)^{-1} is called a pseudo right inverse. We used it for solving underdetermined equations.



In [ ]:
function potential_solve(θ, θ̇, prob::Problem, sys::PickleRick)
    Js = []; fs = []
    for t in prob.tasks 
        ψ = eval(Symbol(t, :_task_map))
        x = ψ(θ, prob)
        J = ForwardDiff.jacobian(σ->ψ(σ, prob), θ)
        ẋ = J*θ̇
        f = potential_eval(x, ẋ, t, prob)
        push!(Js, J); push!(fs, f)
    end
    τ = vec(sum([J'*f for (J, f) in zip(Js, fs)]))  
    return τ
end

Having implemented the `potential_solve` function, we will now pass it into our `dance` function alongside the dancing keypoints we described earlier. The `dance` function computes dance motions for 30 seconds (`horizon` = time\_in\_seconds \* 100) and generates a video of the robot applying those motions.



The motion and video generation takes a few minutes so please be patient with it.



In [ ]:
anim = dance(potential_solve, keypoints; horizon=3000)
gif(anim, "dance.mp4", fps=100)



---

